In [7]:
import re, time,os, datetime
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = Chrome(service=Service(ChromeDriverManager().install()))

In [8]:
excel_file = "Tbill_Returns_2024.xlsx"

try:
	df = pd.read_excel(excel_file,sheet_name="Sheet1")
	dates = list(df["Date"])
except:
	df = pd.DataFrame()
	dates = []

In [9]:
time.sleep(5)
driver.get("https://website.rbi.org.in/web/rbi/press-releases?q=%2291+days%2C+182+days+and+364+days+T-Bill+Auction+Result%3A+Cut+off%22&delta=50")
time.sleep(5)
text = driver.page_source
arr = re.findall('href="https://website.rbi.org.in/web/rbi/-/press-releases/91-days-182-days-and-364-days-t-bill-auction-result-cut-off-.*?"',text)

In [10]:
start = True

for elem in arr:
	if start:
		t = elem[6:-1]
		driver.get(t)
		text1 = driver.page_source

		try:
			date = re.findall("Published on .*2024", text1)[0].split("on")[1]
			date = datetime.datetime.strptime(date, ' %B %d, %Y').strftime('%d %b, %Y')
			print(date)
			if date not in dates:
				with open("tmp.html", "w", encoding="utf-8") as f:
					f.write(text1)
					dfs = pd.read_html("tmp.html")

				d = dfs[2]
				returns = [d.loc[2,2],d.loc[2,3],d.loc[2,4]]
				for i in range(len(returns)):
					try:
						returns[i] = ((returns[i].split(":"))[1])[:-1]
					except:
						returns[i] = "NA"

				df1 = pd.DataFrame({"Press Release ID":[""], "Date":[date], "91 DTB Yield":[returns[0]], "182 DTB Yield":[returns[1]], "364 DTB Yield":[returns[2]]})
				df = pd.concat((df,df1))
				print(date)
			else:
				start = False
		except:
			pass

02 May, 2024
02 May, 2024
24 Apr, 2024


In [11]:
driver.quit()

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date',ascending=True, ignore_index=True)
df['Date'] = df['Date'].dt.strftime('%d %b, %Y')
df.to_excel(excel_file,sheet_name="Sheet1",index = False)

os.remove("tmp.html")